In [3]:
import keras
from keras import backend as K
from keras.layers.core import Lambda, Dense, Masking
from keras.layers import Input, Embedding, LSTM
from keras import Model
from keras.optimizers import Adam
#from keras.engine.topology import Layer

In [1]:
import numpy as np
import os, sys
from importlib import reload

In [11]:
import setsum_layer
reload(setsum_layer)
from setsum_layer import SetSum

In [5]:
n_timestep = 4
n_feature = 3

In [6]:
num_train_examples = 10000
max_train_length = 10

num_test_examples = 10000
min_test_length=5
max_test_length=100
step_test_length=5

In [7]:
X = np.zeros((num_train_examples,max_train_length))
sum_X = np.zeros((num_train_examples))
for i in range(num_train_examples):
    n = np.random.randint(1,max_train_length)
    for j in range(1,n+1):
        X[i,-j] = np.random.randint(1,10)
    sum_X[i] = np.sum(X[i])

In [17]:
inputs = Input(shape=(max_train_length, ))
x = Embedding(10, n_feature, mask_zero=True)(inputs)
x = SetSum(5)(x)
out = Dense(1)(x)
model = Model(inputs=inputs, outputs=out)

In [21]:
model.compile(optimizer='adam', loss='mae')

In [16]:
model.layers

In [18]:
model.layers[2].output_shape

(None, 5)

In [23]:
hist = model.fit(X, sum_X, epochs=5, batch_size=128, shuffle=True, validation_split=0.123456789)

Train on 8765 samples, validate on 1235 samples
Epoch 1/5
8765/8765 [==============================] - 0s 11us/step - loss: 2.0130 - val_loss: 1.4150
Epoch 2/5
8765/8765 [==============================] - 0s 11us/step - loss: 0.8900 - val_loss: 0.2842
Epoch 3/5
8765/8765 [==============================] - 0s 11us/step - loss: 0.1105 - val_loss: 0.0760
Epoch 4/5
8765/8765 [==============================] - 0s 11us/step - loss: 0.0689 - val_loss: 0.0604
Epoch 5/5
8765/8765 [==============================] - 0s 11us/step - loss: 0.0515 - val_loss: 0.0426


In [132]:
model.layers[1].output_mask

<tf.Tensor 'embedding_11/NotEqual:0' shape=(?, 10) dtype=bool>

In [149]:
embed_mat = model.layers[1].get_weights()[0]

In [19]:
model.layers[1].get_weights()[0]

array([[-0.037888  ,  0.02562937, -0.03066218],
       [-0.03654822, -0.01050846,  0.0360505 ],
       [-0.00666766,  0.02890015, -0.0112601 ],
       [ 0.01048945,  0.02720269, -0.03302656],
       [ 0.00501064, -0.02895998,  0.00603358],
       [ 0.02637167,  0.01984045, -0.02812632],
       [-0.00775652, -0.01738523,  0.00421908],
       [-0.03398637, -0.00293621, -0.04882621],
       [ 0.03465379, -0.01851556,  0.00933194],
       [ 0.04919601,  0.00705652, -0.0106264 ]], dtype=float32)

In [24]:
model.layers[1].get_weights()[0]

array([[-0.037888  ,  0.02562937, -0.03066218],
       [-0.03613896, -0.00655953,  0.03242232],
       [ 0.05397358, -0.03181753, -0.07440148],
       [ 0.1365238 , -0.10759641, -0.16054569],
       [ 0.21852352, -0.2678932 , -0.2072292 ],
       [ 0.30838066, -0.30277041, -0.3082521 ],
       [ 0.37281662, -0.461725  , -0.37217689],
       [ 0.41763115, -0.53207082, -0.4946945 ],
       [ 0.56430769, -0.63360906, -0.51339316],
       [ 0.65263557, -0.68722349, -0.60617489]], dtype=float32)

In [150]:
X[:4, :]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  5.,  6.,  3.,  9.],
       [ 0.,  4.,  6.,  1.,  8.,  5.,  6.,  3.,  7.,  8.],
       [ 0.,  3.,  5.,  4.,  9.,  3.,  3.,  4.,  2.,  9.],
       [ 0.,  0.,  0.,  0.,  0.,  3.,  2.,  6.,  4.,  3.]])

In [154]:
embed_mat[2, :]+embed_mat[6, :]+embed_mat[3, :]*2+embed_mat[4, :]+embed_mat[0, :]*5

array([ 0.10518206,  0.27263886,  0.06472662], dtype=float32)

In [151]:
model.predict(X[:4, :])

array([[ 0.07625498, -0.00588461,  0.08142723],
       [ 0.08509673, -0.08664991,  0.18796268],
       [ 0.16553947,  0.11674161,  0.02120311],
       [ 0.06303501,  0.05118224,  0.01142141]], dtype=float32)

In [96]:
def get_deepset_model(max_length):
    input_txt = Input(shape=(max_length,))
    x = Embedding(10, 10, mask_zero=True)(input_txt)
    #x = Dense(30, activation='tanh')(x)
    Adder = Lambda(lambda x: K.sum(x, axis=1), output_shape=(lambda shape: (shape[0], shape[2])))
    x = Adder(x)
    encoded = Dense(1)(x)
    summer = Model(input_txt, encoded)
    adam = Adam(lr=1e-4, epsilon=1e-3)
    summer.compile(optimizer=adam, loss='mae')
    return summer

In [90]:
def get_lstm_model(max_length):
    input_txt = Input(shape=(max_length,))
    x = Embedding(10, 10, mask_zero=True)(input_txt)
    x = LSTM(50)(x)
    encoded = Dense(1)(x)
    summer = Model(input_txt, encoded)
    adam = Adam(lr=1e-4)
    summer.compile(optimizer=adam, loss='mae')
    return summer

In [97]:
model = get_deepset_model(max_train_length)

In [113]:
model = get_lstm_model(max_train_length)

In [117]:
embed_layer = model.layers[1]

In [120]:
embed_layer.output_mask

<tf.Tensor 'embedding_10/NotEqual:0' shape=(?, 10) dtype=bool>

In [98]:
np.linalg.norm(model.layers[1].get_weights()[0], axis=1)

array([ 0.08035915,  0.10173114,  0.07942988,  0.06977315,  0.08350004,
        0.09210253,  0.09292872,  0.06657916,  0.07745726,  0.10820688], dtype=float32)

In [99]:
hist = model.fit(X, sum_X, epochs=10, batch_size=128, shuffle=True, validation_split=0.0123456789)

Train on 9876 samples, validate on 124 samples
Epoch 1/10
9876/9876 [==============================] - 1s 75us/step - loss: 25.4291 - val_loss: 22.3197
Epoch 2/10
9876/9876 [==============================] - 0s 16us/step - loss: 25.1853 - val_loss: 22.0711
Epoch 3/10
9876/9876 [==============================] - 0s 19us/step - loss: 24.9310 - val_loss: 21.8090
Epoch 4/10
9876/9876 [==============================] - 0s 24us/step - loss: 24.6602 - val_loss: 21.5283
Epoch 5/10
9876/9876 [==============================] - 0s 16us/step - loss: 24.3687 - val_loss: 21.2251
Epoch 6/10
9876/9876 [==============================] - 0s 15us/step - loss: 24.0584 - val_loss: 20.9017
Epoch 7/10
9876/9876 [==============================] - 0s 17us/step - loss: 23.7278 - val_loss: 20.5507
Epoch 8/10
9876/9876 [==============================] - 0s 19us/step - loss: 23.3693 - val_loss: 20.1711
Epoch 9/10
9876/9876 [==============================] - 0s 20us/step - loss: 22.9906 - val_loss: 19.7771
Epoch 10

In [100]:
np.linalg.norm(model.layers[1].get_weights()[0], axis=1)

array([ 0.25735557,  0.33120143,  0.26050481,  0.24138205,  0.28021345,
        0.27690503,  0.22618987,  0.24287578,  0.26898557,  0.2789948 ], dtype=float32)

In [168]:
class SumMaskedEmbed(Layer):
    def __init__(self, **kwargs):
        self.supports_masking = True
        super(SumMaskedEmbed, self).__init__(**kwargs)

    def call(self, x, mask=None):
        if mask is not None:
            mask = K.cast(mask, 'float32')
            if not len(mask.shape) == 2:
                print("Mask shape:", mask.shape)
            mask = K.repeat(mask, x.shape[-1])
            mask = K.permute_dimensions(mask, (0, 2, 1))
            masked = mask*x
            return K.sum(masked, axis=1)
        else:
            return K.sum(x, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

    def compute_mask(self, input, input_mask=None):
        return None

In [190]:
from keras import activations
from keras import initializers
from keras import regularizers
from keras import constraints

In [191]:
class SetSum(Layer):
    """The DeepSet operation: apply a dense layer on the last axis, and then do a masked sum on axis=1. 

    # Arguments
        units: Positive integer, dimensionality of the output space.
        activation: Activation function to use
            (see [activations](../activations.md)).
            If you don't specify anything, no activation is applied
            (ie. "linear" activation: `a(x) = x`).
        use_bias: Boolean, whether the layer uses a bias vector.
        kernel_initializer: Initializer for the `kernel` weights matrix
            (see [initializers](../initializers.md)).
        bias_initializer: Initializer for the bias vector
            (see [initializers](../initializers.md)).
        kernel_regularizer: Regularizer function applied to
            the `kernel` weights matrix
            (see [regularizer](../regularizers.md)).
        bias_regularizer: Regularizer function applied to the bias vector
            (see [regularizer](../regularizers.md)).
        activity_regularizer: Regularizer function applied to
            the output of the layer (its "activation").
            (see [regularizer](../regularizers.md)).
        kernel_constraint: Constraint function applied to
            the `kernel` weights matrix
            (see [constraints](../constraints.md)).
        bias_constraint: Constraint function applied to the bias vector
            (see [constraints](../constraints.md)).
    # Input shape
        The input should have the dimensions: (batch, timestep, feature). The last layer should pass a mask with 
        dimension (batch, timestep), which is the default output_mask from Embedding(mask_zero=True)
    # Output shape
        (batch, feature). Note that the timestep dimension is summed out. 
    """
    def __init__(self, units,
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(SetSum, self).__init__(**kwargs)
        self.units = units
        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        self.input_spec = InputSpec(min_ndim=2)
        self.supports_masking = True

    def build(self, input_shape):
        assert len(input_shape) >= 2
        input_dim = input_shape[-1]

        self.kernel = self.add_weight(shape=(input_dim, self.units),
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
        self.input_spec = InputSpec(min_ndim=2, axes={-1: input_dim})
        self.built = True

    def call(self, inputs, mask=None):
        output = K.dot(inputs, self.kernel)
        if self.use_bias:
            output = K.bias_add(output, self.bias, data_format='channels_last')
        if self.activation is not None:
            output = self.activation(output)
            
        if mask is not None:
            mask = K.cast(mask, 'float32')
            if not len(mask.shape) == 2:
                print("Mask shape:", mask.shape)
            mask = K.repeat(mask, units)
            mask = K.permute_dimensions(mask, (0, 2, 1))
            output = output*mask
            
        return K.sum(output, axis=1)

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 3
        assert input_shape[-1]
        return (input_shape[0], self.units)
    
    def compute_mask(self, input, input_mask=None):
        return None

    def get_config(self):
        config = {
            'units': self.units,
            'activation': activations.serialize(self.activation),
            'use_bias': self.use_bias,
            'kernel_initializer': initializers.serialize(self.kernel_initializer),
            'bias_initializer': initializers.serialize(self.bias_initializer),
            'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer': regularizers.serialize(self.bias_regularizer),
            'activity_regularizer': regularizers.serialize(self.activity_regularizer),
            'kernel_constraint': constraints.serialize(self.kernel_constraint),
            'bias_constraint': constraints.serialize(self.bias_constraint)
        }
        base_config = super(Dense, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))